In [1]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
import qutip as qt
from IPython.display import HTML
import scipy as sp

%matplotlib ipympl
plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})
# np.set_printoptions(linewidth=200, formatter={'float': '{:.1e}'.format})

In [15]:
nmax_r = 10
nmax_f = 10

# Symmetric Cc coupling
That is, the inner nodes of the qubits are coupled with Cc, and the outer nodes of the qubits are coupled with Cc to their respective readout lines.
As a result, the renormalization induced by Cc ensures that the qubit and resonator modes are still the symmetric and atisymmetric combination of the fluxes of the upper nodes.

In [42]:
def hamiltonian_qubit_C_qubit(nmax_r, nmax_f, Cc, C = 15, CJ = 3, Csh= 15, Lq = 25, Lr = 10, Δ = 0.1 ):
    
    resonator = sq_ext.KIT_resonator(C = C+Cc,                     Lq = Lq, Lr = Lr, Δ = Δ)
    fluxonium = sq_ext.KIT_fluxonium(C = C+Cc, CJ = CJ, Csh = Csh, Lq = Lq, Lr = Lr, Δ = Δ)
    
    resonator.set_trunc_nums([nmax_r])
    fluxonium.set_trunc_nums([nmax_f])
    
    H_left  = sq_ext.hamiltonian_frc(fluxonium, resonator,Δ)
    H_right = sq_ext.hamiltonian_frc(fluxonium, resonator,Δ)
    
    I_r       = qt.identity(nmax_r)
    I_f       = qt.identity(nmax_f)
    I_H_left  = qt.identity(H_left .dims[0])
    I_H_right = qt.identity(H_right.dims[0])
    
    q_r = qt.tensor(resonator.charge_op(0), I_f)
    q_f = qt.tensor(I_r, fluxonium.charge_op(0))
    
    if Cc>0:
        H = qt.tensor(H_left, I_H_right) + qt.tensor(I_H_left, H_right) + 1/Cc * (qt.tensor(q_r,q_r) + 
                                                                                  qt.tensor(q_f,q_f) + 
                                                                                  qt.tensor(q_r,q_f) + 
                                                                                  qt.tensor(q_f,q_r) )
    else:
        H = qt.tensor(H_left, I_H_right) + qt.tensor(I_H_left, H_right)
        
    return H


In [43]:
H_0 = hamiltonian_qubit_C_qubit(nmax_r, nmax_f, Cc=0)

In [44]:
H   = hamiltonian_qubit_C_qubit(nmax_r, nmax_f, Cc=10)

In [49]:
H_eff = sq_ext.H_eff_p1(H_0,H,6)

In [50]:
print(np.abs(H_eff))

[[13.744782144594 0.000000000000 0.000000000000 0.000000000000 0.183217369285 0.186479855297]
 [0.000000000000 15.371482510944 0.542945295295 0.000000000000 0.000000000000 0.000000000000]
 [0.000000000000 0.542945295295 15.371482510945 0.000000000000 0.000000000000 0.000000000000]
 [0.000000000000 0.000000000000 0.000000000000 16.998182877296 0.000000000000 0.000000000000]
 [0.183217369285 0.000000000000 0.000000000000 0.000000000000 19.106252355198 0.045040238116]
 [0.186479855297 0.000000000000 0.000000000000 0.000000000000 0.045040238116 19.106252355199]]
